In [85]:
import os 
os.chdir('C:\\Users\\harkum\\Documents\\6311\\Software')

In [86]:
import json 

rows = ['SDWN','VBSO_S','VBAT_S','SW_S','BIAS_S','BIAS_F','BSO_F','SW_F_PIN']
cols = ['Battery Emulator (BE)','Voltmeter (VM)','SUPPLY 5V->12V','SUPPLY 3.6V ->11V ->19V','Supply 1.8V']
with open('driver\signals\SignalConfig.json', 'r') as file :
    data = json.load(file)
    
signalMatrix = {}
base_addr = 0x20
for matrix, signalMap in data.items() :
    signalroot = {}
    for row, Signal in enumerate(signalMap.get('rowsHigh')):
        row = row + 1
        for col, instrument in enumerate(signalMap.get('colsHigh')) :
            col = col + 1
            if  1 <= row  < 3:
                device_addr = base_addr + int(matrix) * 4
            elif 3 <= row  < 5:
                device_addr = base_addr + int(matrix) * 4 + 1
            elif 5 <= row  < 7:
                device_addr = base_addr + int(matrix) * 4 + 2
            elif 7 <= row  < 9:
                device_addr = base_addr + int(matrix) * 4 + 3
            signalroot.update(
                {
                    str({"Source":{"High":Signal,"Low":signalMap.get('rowsLow')[row-1] }, "Destination" : {"High":instrument,"Low":signalMap.get('colsLow')[col-1] }}) : {
                    "deviceAddr" : hex(device_addr),
                    "relay":{
                        "row" : row ,
                        "col":col                    
                        }
                    }
                }
            )
    signalMatrix.update({
        matrix : signalroot
    })
print(signalMatrix)
with open('driver/signals/signalroot_matrix.json','w') as file :
    json.dump(signalMatrix ,file,)

{'0': {"{'Source': {'High': 'SDWN', 'Low': 'GND_S'}, 'Destination': {'High': 'Battery Emulator (BE)', 'Low': 'GND (BE-)'}}": {'deviceAddr': '0x20', 'relay': {'row': 1, 'col': 1}}, "{'Source': {'High': 'SDWN', 'Low': 'GND_S'}, 'Destination': {'High': 'Voltmeter (VM)', 'Low': 'GND (VM-)'}}": {'deviceAddr': '0x20', 'relay': {'row': 1, 'col': 2}}, "{'Source': {'High': 'SDWN', 'Low': 'GND_S'}, 'Destination': {'High': 'SUPPLY 5V->12V', 'Low': 'GND'}}": {'deviceAddr': '0x20', 'relay': {'row': 1, 'col': 3}}, "{'Source': {'High': 'SDWN', 'Low': 'GND_S'}, 'Destination': {'High': 'SUPPLY 3.6V ->11V ->19V', 'Low': 'GND'}}": {'deviceAddr': '0x20', 'relay': {'row': 1, 'col': 4}}, "{'Source': {'High': 'SDWN', 'Low': 'GND_S'}, 'Destination': {'High': 'Supply 1.8V', 'Low': 'GND'}}": {'deviceAddr': '0x20', 'relay': {'row': 1, 'col': 5}}, "{'Source': {'High': 'VBSO_S', 'Low': 'GND_S'}, 'Destination': {'High': 'Battery Emulator (BE)', 'Low': 'GND (BE-)'}}": {'deviceAddr': '0x20', 'relay': {'row': 2, 'col'

In [87]:
# evaluvating the literals
from ast import literal_eval
x=literal_eval("{'source': {'High': 'SDWN', 'Low': 'GND_S'}, 'destination': {'High': 'Battery Emulator (BE)', 'Low': 'GND (BE-)'}}")
type(x)

dict

### Signal Identifying pattren

In [89]:
from ast import literal_eval
from pathlib import Path
from typing import List 
import json 
import warnings

def signalConfig(Source:dict, Destination: dict, Signalroot_file : str) -> dict:
    signal_root ={}
    with open(Path(Signalroot_file),'r') as file :
        signals = json.load(file,)
    for  matrix, path_data in signals.items():
        
        for signal, data in path_data.items():
            signal = literal_eval(signal)
            if signal.get('Source').get('High') == Source.get('High') and \
                signal.get('Source').get('Low') == Source.get('Low') :
                if signal.get('Destination').get('High') == Destination.get('High') and \
                    signal.get('Destination').get('Low') == Destination.get('Low') :
                        signal_root = data
            #     else:
            #         warnings.warn('Destination Path is Wrong .......!')
            # else:
            #     warnings.warn('Source Path is Wrong .......!')
    
    return signal_root

source = {'High': 'SDWN', 'Low': 'GND_S'}
destination = {'High': 'Voltmeter (VM)', 'Low': 'GND (VM-)'}
signalConfig(Source=source, Destination=destination,Signalroot_file='driver/signals/signalroot_matrix.json')

{'deviceAddr': '0x20', 'relay': {'row': 1, 'col': 2}}